# Import libraries + dataset

In [1]:
import pandas as pd
import numpy as np
# import sklearn as sk
from datetime import timedelta
# from sklearn import model_selection, tree

In [13]:
filename = 'shot_logs.csv'
data = pd.read_csv(filename)
data = data.dropna() # SHOT_CLOCK originally has NaNs. I wonder why?
print(data.shape)
data.head()

(122502, 21)


,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148
5,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,6,2,8:15,9.1,2,...,18.4,2,missed,"Williams, Deron",101114,2.6,0,0,brian roberts,203148


In [ ]:
# The attributes:

# GAME_ID
# MATCHUP  =  Date - Teams
# LOCATION  =  Home or Away
# W  =  Win/Loss
# FINAL_MARGIN
# SHOT_NUMBER  =  This shot's number (per player, per game)
# PERIOD
# GAME_CLOCK
# SHOT_CLOCK
# DRIBBLES  =  Dribbles before shot
# TOUCH_TIME
# SHOT_DIST
# PTS_TYPE  =  Attempted type of shot
# SHOT_RESULT
# CLOSEST_DEFENDER
# CLOSEST_DEFENDER_PLAYER_ID
# CLOSE_DEF_DIST  =  Distance to the closest defender
# FGM  =  Field Goals Made?
# PTS  =  Points Scored
# player_name
# player_id
        
# Some code I was using to visualize the different values of each column
# for col in data:
#     uniques = data[col].unique()
#     print('{} | # uniques: {}'.format(col, len(uniques)))
#     print('{}\n'.format(uniques[:3]))

# Perform Feature Engineering

In [14]:
# fr https://stackoverflow.com/questions/50308629/python-pandas-column-convert-minutes-to-second
def timeToSeconds(x):
    mins, secs = map(float, x.split(':'))
    td = timedelta(minutes=mins, seconds=secs)
    return td.total_seconds()

In [15]:
# begin to separate the data into features and labels
labels = data['SHOT_RESULT'].map(dict(made=1, missed=0))

### Try out keeping different features by running different cells
The model code should be based solely on feature_data to easily test different feature extractions

In [ ]:
# drop_features = ['SHOT_RESULT', 'MATCHUP', 'GAME_ID', 'CLOSEST_DEFENDER','player_name', 'LOCATION', 'W', 'PTS', 'FGM']


In [ ]:
# There are a lot more features we want to get rid of than use -- let's specify which ones we want to keep
# desired_features = ['SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME', 'SHOT_DIST', 'PTS_TYPE', 'CLOSE_DEF_DIST', 'LOCATION', 'W']

There are 21 features in our dataset, both categorical and numerical. The 'pts', and 'fgm' features are redundant for us, because they are non-zero (in the case of 'pts') and increasing (in the case of 'fgm') for made shots and zero and non-inreasing otherwise, so they essentially describes the 'shot_result' attribute we chose to be our label. 

Additionally, there are a number of features such as 'player_id', 'player_name', 'closest_defender_player_id', 'closest_defender', 'location', and 'matchup' that seem like they might be valuable (i.e. some defenders are better and the resulting shot likelihood would be lower), but not valuable enough to enhance the accuracy of our classifier.

There are also features that describe the context of the game the shot was taken in such as 'game_clock', 'period', 'final_margin', 'W', 'game_id' that are not pertinent to the specific shot we are trying to predict. 

There is a third class of features that we dropped originally that describe the game a specific shooter is having, such as 'shot_number' and 'FGM' that describe which shot of the game it is for a given player.

This left us with six features that describe only the specific situation a shot was taken in:

1. shot_clock: The amount of time remaining in a possession. It figures that if there is only a small amount of time left, a player might rush to get a shot off.

2. touch_time: The amount of time the ball is in our specific shooter's posession. It seems that the longer the ball is in a player's possession, the less likely they are to score because they didn't have a clear shot when they recieved the ball.

3. dribbles: The amount of dribbles the player takes before shooting. Dribbling a lot might mean the player did not have a clear shot when he recieved the ball, resulting in a less likely shot. 

4. pts_type: Whether the shot was a 2 or 3 point field goal. It figures that because 3-pointers are taken farther from the basket, they might be harder to score.

5. shot_dist: How far from the basket the shot was taken. The closer a player is to the basket, the easier it is to score.

6. close_def_dist: How far the closest defender is. The closer the nearest defender, the more contested and harder to make a shot becomes. 

In [16]:
desired_features = ['SHOT_CLOCK', 'TOUCH_TIME', 'SHOT_DIST', 'CLOSE_DEF_DIST']

In [ ]:
# desired_features = [?]

In [17]:
# feature_data = data.drop(drop_features, axis=1)
feature_data = data.loc[:, desired_features]

# Reads game clock into seconds
# feature_data['GAME_CLOCK'] = data['GAME_CLOCK'].apply(timeToSeconds)

# One hot encodes any remaining categorical variables (like Location or W)
feature_data = pd.get_dummies(feature_data)

feature_data.head()

,SHOT_CLOCK,TOUCH_TIME,SHOT_DIST,CLOSE_DEF_DIST
0,10.8,1.9,7.7,1.3
1,3.4,0.8,28.2,6.1
3,10.3,1.9,17.2,3.4
4,10.9,2.7,3.7,1.1
5,9.1,4.4,18.4,2.6


# === Models ===

In [18]:
import sklearn as sk

# Tools
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import Pipeline

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Parallelism
from sklearn.utils import parallel_backend


## Decision Tree

In [19]:
# Essentially a playground for trying different parameters of decision trees
# These should probably be run with the cross validation of the cell below tbh, but these are just examples
train_features, test_features, train_labels, test_labels = sk.model_selection.train_test_split(feature_data, labels, test_size=0.20)

# use gini
classifier = DecisionTreeClassifier(criterion='gini')
classifier = classifier.fit(train_features, train_labels)
predictions = classifier.predict(test_features)
print('Accuracy on test data using Gini as split criterion is:', (sk.metrics.accuracy_score(test_labels, predictions)))
print()

# max depth
depth=4
classifier = DecisionTreeClassifier(criterion='entropy', max_depth=depth)
classifier = classifier.fit(train_features, train_labels)
predictions = classifier.predict(test_features)
print('Accuracy on test data with max tree depth', depth, 'is:', (sk.metrics.accuracy_score(test_labels, predictions)))
print()

# minimum samples in a split set to 5
classifier = DecisionTreeClassifier(criterion='entropy', min_samples_split=5)
classifier = classifier.fit(train_features, train_labels)
predictions = classifier.predict(test_features)
print('Accuracy on test data with max tree minimum sample split 5 is:', (sk.metrics.accuracy_score(test_labels, predictions)))

Accuracy on test data using Gini as split criterion is: 0.5352434594506347

Accuracy on test data with max tree depth 4 is: 0.6116893188033141

Accuracy on test data with max tree minimum sample split 5 is: 0.5393657401738704


In [20]:
# Run cross validation with chosen hyperparameters

classifier = DecisionTreeClassifier(criterion='entropy', max_depth=10)
with parallel_backend('loky'):
    scores = cross_val_score(classifier, feature_data, labels, cv=5)
                                           
print("Average Accuracy:", scores.mean()*100)

Average Accuracy: 61.154099599433266


In [22]:
# Inner loop of nested cross validation - determine what are likely the best hyperparams

classifier = DecisionTreeClassifier()
params = {"max_depth": [5, 10, 15, 20],
          "min_samples_leaf": [5, 10, 15, 20],
          "max_features": [3, 4]}

grid_search = GridSearchCV(classifier, params, cv=5, scoring='accuracy')
with parallel_backend('loky'):
    grid_search.fit(feature_data, labels)

print(grid_search.best_params_)
print("Average Accuracy:", grid_search.best_score_*100)

{'max_depth': 5, 'max_features': 4, 'min_samples_leaf': 5}
Average Accuracy: 61.434910228570416


In [23]:
# The Real Deal: Nested Cross Validation

with parallel_backend('loky'):
    scores = cross_val_score(grid_search, feature_data, labels, cv=10) 

print(grid_search.best_params_)
print("Average Accuracy:", scores.mean()*100)

KeyboardInterrupt: ignored

## K Nearest Neighbors
Here we will explore the K nearest neighbors technique. This method is 
used to determine association based on the K nearest neighbors surrounding 
a data point. Based on this association, the prediction can be made based 
on classification of its neighbors. 

In [24]:
scaler = sk.preprocessing.StandardScaler()
PCA = sk.decomposition.PCA()
knn = sk.neighbors.KNeighborsClassifier(n_neighbors=100)
pipe = Pipeline([('scaler' , scaler ), ('pca', PCA), ('knn', knn) ] )

with parallel_backend('loky'):
    accuracies = cross_val_score( pipe, feature_data, labels, cv=5)
    
print("Accuracies: ", accuracies.mean()*100)

Accuracies:  61.15573091977087


In [26]:
param_grid = {
    #'pca__n_components': list(range(5, 10)),
    'knn__n_neighbors': list(range(80, 120, 5))
}
gs = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')

with parallel_backend('loky'):
    gs.fit(feature_data, labels)

print(gs.best_params_)
print(gs.best_score_)

{'knn__n_neighbors': 110}
0.6117858822921847


In [ ]:
with parallel_backend('loky'):
    accuracies = cross_val_score( gs, feature_data, labels, cv=5)
print("Accuracies: ", accuracies.mean()*100)

Accuracies:  58.034969434816894


## Naive Bayes

In [ ]:
# From the HW
classifier = GaussianNB()

# Cross validation accuracy
scores = cross_val_score(classifier, feature_data, labels, cv=10) 
print("Average Accuracy:", scores.mean()*100)

duration = 0.3098483085632324
Average Accuracy: 57.91334005220732


In [ ]:
# Cross validation accuracy + other metrics
predictions = cross_val_predict(classifier, feature_data, labels, cv=10)

matrix = sk.metrics.confusion_matrix(labels, predictions)
report = sk.metrics.classification_report(labels, predictions)
print(report)

duration = 0.3035438060760498
              precision    recall  f1-score   support

           0       0.62      0.59      0.60     66622
           1       0.54      0.57      0.55     55880

    accuracy                           0.58    122502
   macro avg       0.58      0.58      0.58    122502
weighted avg       0.58      0.58      0.58    122502



## SVM

In [ ]:
scaler = sk.preprocessing.StandardScaler()
pca = sk.decomposition.PCA()
svc = SVC()
pipeline = Pipeline(steps=[('scaler', scaler), ('PCA', pca), ('SVC', svc)])

param_grid = {
    #'PCA__n_components': list(range(2, 6)),
    'SVC__kernel': ['linear', 'rbf', 'poly']
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3)

with parallel_backend('loky'):
    predictions = cross_val_predict(grid_search, feature_data, labels, cv=3)

avg_accuracy = sk.metrics.accuracy_score(labels, predictions)
print("Average Accuracy:", avg_accuracy*100)
report = sk.metrics.classification_report(labels, predictions)
print(report)

## Neural Network

## Random Forest

Scratch Notes I had on Classification Models in case someone finds them useful:

#### Decision Trees
- Choose Tree Depth
- Different ways to split categorical (half and half, one and the rest, ...)
- Random Forests

#### K Nearest Neighbors
- Choose K
- Data Normalization
- Curse of Dimensionality
- Class imbalance worries
- Feature selection very important

#### Naive Bayes
-    

#### SVM
- Choose cost for soft-margin
- Choose kernel functions
- Data Normalization
- Onehot encoding for categorical

#### Ensemble Techniques
- Bagging (give each model a sample of data)
- Boosting (train models iteratively on the subset of data the last one didn't do well on)
- Stacking (use one model's predictions as part of input to next model)

#### Neural Network
- Data Normalization
- Onehot encoding for categorical
- Takes a lot of data to train